<a href="https://colab.research.google.com/github/ChandraMaddikunta/Week3-lab-Fashion-MNIST/blob/master/Week_3_Lab_Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this lab we will train a neural network model to classify images of clothing. 

The lab is designed to be run using Google Colab, but also can be run as a normal jupyter notebook with minor modifications.

This lab uses [tf.keras](https://www.tensorflow.org/guide/keras), a high-level API to build and train models in TensorFlow.

**Chandra Maddikunta**
> EAI 6000

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Import the Fashion MNIST dataset

In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

Examine the dataset size.
1. What are the sizes of each image? 
2. How many categories are there?
3. How many images are there in the test and training sets?
4. What are the range of values for the images? 

In [0]:
train_images[0].shape 

1) Size of each image is 28 by 28 pixels

In [0]:
len(class_names)

2) There are 10 categories

In [0]:
train_labels.shape

3) There are 60,000 images in training set

In [0]:
test_labels.shape

3) There are 10,000 images in test set

In [0]:
#train_images[0]
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

In [0]:
train_images.max()

In [0]:
train_images.min()

4) From inspecting the above image, max and min values we can say that range of values for the images are from 0 to 255

Preprocess the data by scaling the pixel values of all images to fall between 0 and 1.

In [0]:
train_images = train_images / 255.0

test_images = test_images / 255.0

To verify that the data is in the correct format  display the first 10 images from the *training set* and display the class name below each image.


In [0]:
plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Build the model

Using Keras, build a neural network with a single hidden layer consisting of 64 neurons. Remember that you will either have to reshape the image data into 1D vectors, or use a flatten layer at the input.
1. Set up the model layers
2. Compile the model 
3. Print the model summary

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10)
])

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [0]:
model.summary()

## Train the model

Train the model on the training images for 20 epochs.




In [0]:
model.fit(train_images, train_labels, epochs=20)

Evaluate the accuracy of the model on the test dataset. How does it compare with the training accuracy? Is it likely overfitting or underfitting?

In [0]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

Test accuracy is slightly less than training accuracy so we can say that the model is slightly overfitting but the difference is less than 0.05 so it is a pretty good model, since both training and testing accuracies are pretty high the model is not underfitting.

### Make predictions

The outputs layer of the model contains predictions about the classification for each image. 

In [0]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
class_predictions = model.predict_classes(test_images)
prediction_probabilities = probability_model.predict(test_images)

In [0]:
class_predictions.shape

In [0]:
prediction_probabilities.shape

In [0]:
prediction_probabilities[0]

Examine the shape of the variable 'prediction_probabilities'. How is it different from class_predictions? What do the dimensions correspond to?

prediction_probabilities has (10000, 10) shape which means it has 10000 rows and 10 columns where 10000 rows means 10000 testing images and 10 columns correspond to probability of image belonging to the 10 classes each. class_predictions has (10000,) shape which means it has 10000 rows and 1 column where 10000 rows means 10000 testing images and 1 columns correspond to the predicted class for that image.

The following functions plot the image and a bar graph of the predicted probabilities. Using these functions, make a plot of the first 25 test images that were incorrectly classified along with their predicted probabilities. Interpret these images - what sort of images tend to be misclassified?

In [0]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(np.arange(10),list(class_names),rotation=90)
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)


  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [0]:
def plot_misimage(predictions_array, true_label, img):
  count = 0
  plt.figure(figsize=(20, 10))
  for i in range(10000):
      #img = img[i]
      predicted_label = np.argmax(predictions_array[i])
      if predicted_label == true_label[i]:
        color = 'blue'
      else:
        color = 'red'
      if color == 'red':
          plt.subplot(5,10,2*count+1)
          plt.grid(False)
          plt.xticks([])
          plt.yticks([])
          plt.imshow(img[i], cmap=plt.cm.binary)
          plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                    100*np.max(predictions_array[i]),
                                    class_names[true_label[i]]),
                                    color=color)
          plt.subplot(5,10,2*count+2)
          plt.grid(False)
          #range(10)
          #plt.xticks(np.arange(10),list(class_names),rotation=90)
          plt.xticks(range(10))
          plt.yticks([])
          thisplot = plt.bar(range(10), predictions_array[i], color="#777777")
          plt.ylim([0, 1])
          predicted_label = np.argmax(predictions_array[i])


          thisplot[predicted_label].set_color('red')
          thisplot[true_label[i]].set_color('blue')
          count = count+1
      if count == 25:
         break

In [0]:
plot_misimage(prediction_probabilities, test_labels, test_images)
plt.show()

The above plot shows the first 25 misclassified images along with their prediction probability values. From the plot we can see that images which are misclassified are those that have high resemblence to other classes like pullover as shirt, Ankle boot as sneaker, etc these are expected errors. There are no misclassifications between classes with huge differences like shirt as sneaker, sandal as trouser, etc.

Plot a confusion matrix of the predictions. Which misclassifications are the most common?

In [0]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_labels,class_predictions)
fig, ax = plt.subplots(figsize= (12,6))
im = ax.imshow(cm)
ax.set_xticks(np.arange(10)), ax.set_yticks(np.arange(10))
ax.set_xticklabels(list(class_names), rotation=45, ha="right")
ax.set_yticklabels(list(class_names))
ax.set_ylabel('True')
ax.set_xlabel('Predicted')
for i in range(100):
    ax.text(int(i/10),i%10,cm[i%10,int(i/10)], ha="center", va="center", color="w")

From the Confusion matrix above we can see that most common misclassifications are T-shirt/top missclassified as Shirt follwed by Pullover missclassified as Coat, Coat missclassified as Pullover, Shirt missclassified as T-shirt/top, etc. We can see that most of the missclassifications are between classes which have high resemblence between their outer apperances.